In [105]:
import numpy as np
import pandas as pd
np.set_printoptions(threshold=123456789)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import math, random
from collections import Counter
from itertools import product
from copy import deepcopy

from rdkit import Chem
from rdkit.Chem import AllChem
import os

In [90]:
def filter_columns(fps_np, print_ = True):
    fps = pd.DataFrame(fps_np)
    fps_drop0 = fps.loc[:, ~(fps == 0).all()]
    fps_drop1 = fps_drop0.loc[:, ~(fps_drop0 == 1).all()]
    fps_nondup = fps_drop1.T.drop_duplicates().T
    if print_: print('FPs shape: original = {}, filter0 = {}, filter1 = {}, drop_duplicate_column = {}'.format(
                     fps.shape, fps_drop0.shape, fps_drop1.shape, fps_nondup.shape))
    return fps_nondup

In [300]:
def generate_fp_with_id(pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/3_drop_repeat_smiles_from2folder/', 
                        target = 'GPCR',
                        save_pwd = '/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/',
                        radius = 2):

    dti_data = pd.read_csv(pwd + 'v2_' + target + '_updated_drug_smiles_ids_drop_repeated.csv')
    drug_id_smi = dti_data.drop_duplicates(['drug_id', 'smiles']).reset_index(drop = True)[['drug_id', 'smiles']]
    print(dti_data.shape, drug_id_smi.shape)
    
    rd_mols = [Chem.MolFromSmiles(smiles) for smiles in drug_id_smi['smiles']]
    assert len(rd_mols) == drug_id_smi.shape[0]

    rdkit_morgan_mat = [list(AllChem.GetMorganFingerprintAsBitVect(mol, radius = radius, nBits = 2048)) for mol in rd_mols]
    assert len(rdkit_morgan_mat) == len(rd_mols)
    assert len(rdkit_morgan_mat[0]) == 2048

    fps_nondup = filter_columns(rdkit_morgan_mat)
    print('After drop same fps = ', fps_nondup.drop_duplicates(keep = 'first').shape)
    fps_with_smi_id = pd.concat([drug_id_smi, fps_nondup], axis = 1)
    fps_with_smi_id.to_csv(save_pwd + target +'_U_rdkit_morgan' + str(radius) +'_with_id.csv', index = False)
    return dti_data, fps_with_smi_id

In [306]:
def find_same_fps_id(fps_with_smi_id, dti_data):
    duplicate = fps_with_smi_id[fps_with_smi_id.iloc[:, -2018:].duplicated(keep = False) == True]
#     print('Duplicate fps = ', duplicate.shape)
    duplicate = duplicate.sort_values(list(duplicate.columns[2:]))
#     print('Duplicate fps = ', duplicate.shape)
    print('Duplicate fps = {}, ratio dup/all = {}'.format(duplicate.shape, duplicate.shape[0]/fps_with_smi_id.shape[0]), end = ',')
    duplicate = duplicate.groupby(list(duplicate.columns[2:])).sum()
#     print(duplicate.columns)
    duplicate_dict = {}
    for idx, ids in enumerate(duplicate['drug_id']):
        ids = ids.replace('D', ',D')[1:].split(',')
        duplicate_dict[idx] = {}
        for id_ in ids:
            duplicate_dict[idx][id_] = list(dti_data[dti_data['drug_id'] == id_]['hsa_id'])
    print(len(duplicate_dict))
    return duplicate_dict

# radius = 2

In [142]:
gpcr_dti_data, gpcr_fps_with_smi_id = generate_fp_with_id(pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/3_drop_repeat_smiles_from2folder/', 
                        target = 'GPCR',
                        save_pwd = '/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/')
gpcr_fps_with_smi_id.head()

(5383, 3) (1680, 2)
FPs shape: original = (1680, 2048), filter0 = (1680, 2018), filter1 = (1680, 2018), drop_duplicate_column = (1680, 2018)
(1671, 2018)


drug_id                                             smiles  0  1  2  3  4  \
0  D06000                  C#CCOC1CN2CCC1CC2.O=C(O)C=CC(=O)O  0  0  0  0  0   
1  D03102                                 CNCCc1ccccn1.Cl.Cl  0  0  0  0  0   
2  D08517  Cc1cc2c(cc1CC(=O)c1sccc1S(=O)(=O)Nc1onc(C)c1Cl...  0  0  0  0  0   
3  D07810  CC(=O)Oc1ccc2c3c1OC1C(OC(C)=O)C=CC4C(C2)N(C)CC...  0  0  0  0  0   
4  D09752                      COC(=O)CCC[N+](C)(C)C.O.[Cl-]  0  0  0  0  0   

   5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  \
0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
1  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
2  0  0  0  0  0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   
3  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
4  0  0  0  0  0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   

   25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  \
0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   
1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
4   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

   44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  \
0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
1   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   
2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
4   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

   63  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  \
0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   
1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   
2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   
3   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
4   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   

   83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  \
0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    0   
1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    0   
2   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0    0   
3   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0    0   
4   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    0   

   101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  \
0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
1    1    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
2    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
3    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
4    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   

   116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  \
0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
1    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
2    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
3    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
4    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   

   131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  \
0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   

In [278]:
ic_dti_data, ic_fps_with_smi_id = generate_fp_with_id(pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/3_drop_repeat_smiles_from2folder/', 
                        target = 'IC',
                        save_pwd = '/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/')
# ic_fps_with_smi_id.head()
find_same_fps_id(ic_fps_with_smi_id, ic_dti_data)

(6385, 3) (765, 2)
FPs shape: original = (765, 2048), filter0 = (765, 1913), filter1 = (765, 1913), drop_duplicate_column = (765, 1857)
After drop same fps =  (763, 1857)
Duplicate fps =  (0, 1859)
Duplicate fps =  (0, 1859)
Index([], dtype='object')


KeyError: 'drug_id'

In [321]:
17/2048

0.00830078125

In [279]:
e_dti_data, e_fps_with_smi_id = generate_fp_with_id(pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/3_drop_repeat_smiles_from2folder/', 
                        target = 'E',
                        save_pwd = '/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/')
# e_fps_with_smi_id.head()
find_same_fps_id(e_fps_with_smi_id, e_dti_data)

(7371, 3) (1777, 2)
FPs shape: original = (1777, 2048), filter0 = (1777, 2046), filter1 = (1777, 2046), drop_duplicate_column = (1777, 2046)
After drop same fps =  (1765, 2046)
Duplicate fps =  (23, 2048)
Duplicate fps =  (23, 2048)
Index(['drug_id', 'smiles'], dtype='object')


{0: {'D00119': ['hsa26279'],
  'D05341': ['hsa1558', 'hsa5538', 'hsa3906'],
  'DB03017': ['hsa128', 'hsa5320', 'hsa4057', 'hsa26279']},
 1: {'D01968': ['hsa2224', 'hsa9453'], 'D10515': ['hsa2224']},
 2: {'D03234': ['hsa2224'], 'D11570': ['hsa2224']},
 3: {'D01984': ['hsa495', 'hsa496', 'hsa23576'],
  'D11114': ['hsa495', 'hsa496']},
 4: {'D02451': ['hsa1588',
   'hsa1543',
   'hsa1544',
   'hsa1545',
   'hsa1548',
   'hsa1549',
   'hsa1551',
   'hsa1553',
   'hsa1555',
   'hsa1557',
   'hsa1558',
   'hsa1559',
   'hsa1562',
   'hsa1565',
   'hsa1571',
   'hsa1572',
   'hsa1573',
   'hsa1576',
   'hsa1577',
   'hsa1580',
   'hsa199974',
   'hsa260293',
   'hsa29785',
   'hsa64816'],
  'D03778': ['hsa1588',
   'hsa1543',
   'hsa1544',
   'hsa1545',
   'hsa1548',
   'hsa1549',
   'hsa1551',
   'hsa1553',
   'hsa1555',
   'hsa1557',
   'hsa1558',
   'hsa1559',
   'hsa1562',
   'hsa1565',
   'hsa1571',
   'hsa1572',
   'hsa1573',
   'hsa1576',
   'hsa1577',
   'hsa1580',
   'hsa199974',
   

In [222]:
gpcr_duplicate = gpcr_fps_with_smi_id[gpcr_fps_with_smi_id.iloc[:, -2018:].duplicated(keep = False) == True]
gpcr_duplicate = gpcr_duplicate.sort_values(list(gpcr_duplicate.columns[2:]))
print(gpcr_duplicate.shape)
gpcr_duplicate = gpcr_duplicate.groupby(list(gpcr_duplicate.columns[2:])).sum()
gpcr_duplicate

(17, 2020)


drug_id  \
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557 558 559 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576 577 578 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 600 601 602 603 604 605 606 607 608 609 610 611 612 613 614 615 616 617 618 619 620 621 622 623 624 625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 698 699 701 702 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848 849 850 851 852 854 855 856 857 858 859 860 861 862 863 864 865 866 867 868 869 870 871 872 873 874 875 876 878 879 880 881 882 883 884 885 886 887 888 889 890 891 892 893 894 895 896 897 898 899 900 901 902 903 904 905 906 908 909 910 911 912 913 914 915 916 917 918 919 920 921 922 924 925 926 927 928 929 930 931 932 933 934 935 936 937 939 940 941 942 944 945 946 947 948 949 950 951 952 953 954 955 956 957 958 959 960 961 962 963 964 965 966 967 968 969 970 971 972 973 974 975 976 977 978 979 980 981 982 983 984 985 986 987 988 989 990 991 992 993 994 995 996 997 998 999 1000 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1018 1019 1020 1021 1022 1023 1024 1025 1026 1027 1028 1029 1031 103

In [258]:
gpcr_duplicate_dict = {}
for idx, ids in enumerate(gpcr_duplicate['drug_id']):
    ids = ids.replace('D', ',D')[1:].split(',')
    gpcr_duplicate_dict[idx] = {}
    for id_ in ids:
#         print(idx, ids, id_,gpcr_duplicate_dict)
        gpcr_duplicate_dict[idx][id_] = list(gpcr_dti_data[gpcr_dti_data['drug_id'] == id_]['hsa_id'])
gpcr_duplicate_dict

{0: {'D02213': ['hsa1813', 'hsa3360', 'hsa1128'],
  'D05008': ['hsa1813', 'hsa3360', 'hsa1128']},
 1: {'D00792': ['hsa1813', 'hsa1812', 'hsa3356', 'hsa3358'],
  'D00793': ['hsa1813', 'hsa1812', 'hsa3356', 'hsa3358']},
 2: {'D08341': ['hsa1813'], 'D08342': ['hsa1813']},
 3: {'D08425': ['hsa1131', 'hsa1128', 'hsa1129', 'hsa1132', 'hsa1133'],
  'D08638': ['hsa1131', 'hsa1128', 'hsa1129', 'hsa1132', 'hsa1133'],
  'DB00942': ['hsa1128']},
 4: {'D00782': ['hsa1128', 'hsa1129', 'hsa1131', 'hsa1132', 'hsa1133'],
  'D00787': ['hsa1128', 'hsa1129', 'hsa1131', 'hsa1132', 'hsa1133']},
 5: {'D00719': ['hsa1128', 'hsa1129', 'hsa1131', 'hsa1132', 'hsa1133'],
  'D02069': ['hsa1128', 'hsa1129', 'hsa1131', 'hsa1132', 'hsa1133']},
 6: {'D00683': ['hsa154'], 'D04701': ['hsa154']},
 7: {'D03694': ['hsa2798'], 'D08113': ['hsa2798']}}

# radiu 4

In [307]:
gpcr_dti_data, gpcr_mg4_with_smi_id = generate_fp_with_id(
                        pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/3_drop_repeat_smiles_from2folder/', 
                        target = 'GPCR',
                        save_pwd = '/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/',
                        radius = 4)
gpcr_mg4_duplicate_drug_dict = find_same_fps_id(gpcr_mg4_with_smi_id, gpcr_dti_data)

(5383, 3) (1680, 2)
FPs shape: original = (1680, 2048), filter0 = (1680, 2048), filter1 = (1680, 2048), drop_duplicate_column = (1680, 2048)
After drop same fps =  (1677, 2048)
Duplicate fps = (6, 2050), ratio dup/all = 0.0035714285714285713,3


In [308]:
ic_dti_data, ic_mg4_with_smi_id = generate_fp_with_id(
                        pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/3_drop_repeat_smiles_from2folder/', 
                        target = 'IC',
                        save_pwd = '/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/',
                        radius = 4)
ic_mg4_duplicate_drug_dict = find_same_fps_id(ic_mg4_with_smi_id, ic_dti_data)

(6385, 3) (765, 2)
FPs shape: original = (765, 2048), filter0 = (765, 2048), filter1 = (765, 2048), drop_duplicate_column = (765, 2048)
After drop same fps =  (763, 2048)
Duplicate fps = (4, 2050), ratio dup/all = 0.00522875816993464,2


In [309]:
e_dti_data, e_mg4_with_smi_id = generate_fp_with_id(
                        pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/3_drop_repeat_smiles_from2folder/', 
                        target = 'E',
                        save_pwd = '/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/',
                        radius = 4)
e_mg4_duplicate_drug_dict = find_same_fps_id(e_mg4_with_smi_id, e_dti_data)

(7371, 3) (1777, 2)
FPs shape: original = (1777, 2048), filter0 = (1777, 2048), filter1 = (1777, 2048), drop_duplicate_column = (1777, 2048)
After drop same fps =  (1763, 2048)
Duplicate fps = (27, 2050), ratio dup/all = 0.015194147439504783,13


# multilabel_XY

In [289]:
def generate_dti_data_with_drug_feature(dti_data, drug_feature):

    label_num = len(set(dti_data['hsa_id']))
    
    data_target_label = pd.concat([dti_data, pd.get_dummies(dti_data['hsa_id'])], axis = 1).groupby(by = 'drug_id').sum().reset_index()
    data_target_label = pd.merge(drug_feature, data_target_label, on = 'drug_id').sample(frac = 1.0, random_state = 1231).reset_index(drop = True)
    print(data_target_label.shape)
    x_target_label = data_target_label.iloc[:, 2:-label_num] # 2 columns:smiles, drug_id
    y_target_label = data_target_label.iloc[:, -label_num:]
   
    assert x_target_label.shape[1] == drug_feature.shape[1] - 2 # 2 columns:smiles, drug_id
    assert y_target_label.shape[1] == label_num
    print(x_target_label.shape, y_target_label.shape)
    x_t_np = np.array(x_target_label)
    y_t_np = np.array(y_target_label)
    print('X.shape = {}, Y.shape = {}'.format(x_t_np.shape, y_t_np.shape))
    
    return  data_target_label['drug_id'], y_target_label.columns, data_target_label, x_t_np, y_t_np

In [144]:
drugs, labels, all_pd, gpcr_x_t_np, gpcr_y_t_np = generate_dti_data_with_drug_feature(gpcr_dti_data, gpcr_fps_with_smi_id)
assert (labels.tolist() == all_pd.columns[-156:]).all()

pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'GPCR_U_T_xy_np_rdkit_morgan_radius2.npz', x_t_np = gpcr_x_t_np, y_t_np = gpcr_y_t_np)

labels.tolist()


(1680, 2176)
(1680, 2018) (1680, 156)
X.shape = (1680, 2018), Y.shape = (1680, 156)


['hsa10161',
 'hsa10203',
 'hsa10266',
 'hsa10267',
 'hsa10268',
 'hsa10800',
 'hsa10803',
 'hsa11251',
 'hsa11255',
 'hsa1128',
 'hsa1129',
 'hsa1131',
 'hsa1132',
 'hsa1133',
 'hsa1234',
 'hsa1241',
 'hsa1268',
 'hsa1269',
 'hsa134',
 'hsa134864',
 'hsa135',
 'hsa136',
 'hsa1394',
 'hsa140',
 'hsa146',
 'hsa147',
 'hsa148',
 'hsa150',
 'hsa151',
 'hsa151306',
 'hsa152',
 'hsa153',
 'hsa154',
 'hsa155',
 'hsa1812',
 'hsa1813',
 'hsa1814',
 'hsa1815',
 'hsa1816',
 'hsa185',
 'hsa186',
 'hsa1901',
 'hsa1903',
 'hsa1909',
 'hsa1910',
 'hsa2149',
 'hsa222545',
 'hsa2357',
 'hsa23620',
 'hsa2550',
 'hsa2692',
 'hsa2693',
 'hsa2740',
 'hsa2798',
 'hsa2835',
 'hsa2841',
 'hsa2846',
 'hsa2847',
 'hsa2852',
 'hsa2859',
 'hsa2862',
 'hsa2864',
 'hsa2911',
 'hsa2912',
 'hsa2913',
 'hsa2914',
 'hsa2915',
 'hsa2916',
 'hsa2917',
 'hsa2918',
 'hsa3061',
 'hsa3062',
 'hsa3269',
 'hsa3274',
 'hsa3350',
 'hsa3351',
 'hsa3352',
 'hsa3354',
 'hsa3355',
 'hsa3356',
 'hsa3357',
 'hsa3358',
 'hsa3360',
 'h

In [281]:
ic_labels, ic_all_pd, ic_x_t_np, ic_y_t_np = generate_dti_data_with_drug_feature(ic_dti_data, ic_fps_with_smi_id)
assert (ic_labels.tolist() == ic_all_pd.columns[-238:]).all()

pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'IC_U_T_xy_np_rdkit_morgan_radius2.npz', x_t_np = ic_x_t_np, y_t_np = ic_y_t_np)

ic_labels.tolist()


(765, 2097)
(765, 1857) (765, 238)
X.shape = (765, 1857), Y.shape = (765, 238)


['hsa10008',
 'hsa10021',
 'hsa10060',
 'hsa10242',
 'hsa10368',
 'hsa10369',
 'hsa1080',
 'hsa11254',
 'hsa11280',
 'hsa1134',
 'hsa1135',
 'hsa1136',
 'hsa1137',
 'hsa1138',
 'hsa1139',
 'hsa1140',
 'hsa1141',
 'hsa1142',
 'hsa1143',
 'hsa1144',
 'hsa1145',
 'hsa1146',
 'hsa116443',
 'hsa116444',
 'hsa1179',
 'hsa1181',
 'hsa1187',
 'hsa1192',
 'hsa1259',
 'hsa162514',
 'hsa169522',
 'hsa170572',
 'hsa200909',
 'hsa200959',
 'hsa22802',
 'hsa22953',
 'hsa23630',
 'hsa23704',
 'hsa2554',
 'hsa2555',
 'hsa2556',
 'hsa2557',
 'hsa2558',
 'hsa2559',
 'hsa2560',
 'hsa2561',
 'hsa2562',
 'hsa2563',
 'hsa2564',
 'hsa2565',
 'hsa2566',
 'hsa2567',
 'hsa2568',
 'hsa2569',
 'hsa2570',
 'hsa26251',
 'hsa27012',
 'hsa27091',
 'hsa27092',
 'hsa27094',
 'hsa27133',
 'hsa27345',
 'hsa2741',
 'hsa2742',
 'hsa2743',
 'hsa285242',
 'hsa2890',
 'hsa2891',
 'hsa2892',
 'hsa2893',
 'hsa2894',
 'hsa2895',
 'hsa2897',
 'hsa2898',
 'hsa2899',
 'hsa2900',
 'hsa2901',
 'hsa2902',
 'hsa2903',
 'hsa2904',
 'hsa

In [283]:
e_labels, e_all_pd, e_x_t_np, e_y_t_np = generate_dti_data_with_drug_feature(e_dti_data, e_fps_with_smi_id)
assert (e_labels.tolist() == e_all_pd.columns[-1411:]).all()

pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'E_U_T_xy_np_rdkit_morgan_radius2.npz', x_t_np = e_x_t_np, y_t_np = e_y_t_np)

e_labels.tolist()


(1777, 3459)
(1777, 2046) (1777, 1411)
X.shape = (1777, 2046), Y.shape = (1777, 1411)


['hsa10',
 'hsa100',
 'hsa10000',
 'hsa10013',
 'hsa100130827',
 'hsa10014',
 'hsa10038',
 'hsa10039',
 'hsa100526835',
 'hsa100533105',
 'hsa10056',
 'hsa100861540',
 'hsa10105',
 'hsa10114',
 'hsa10135',
 'hsa10157',
 'hsa1017',
 'hsa1018',
 'hsa10188',
 'hsa1019',
 'hsa102',
 'hsa1020',
 'hsa1021',
 'hsa1022',
 'hsa1024',
 'hsa10243',
 'hsa10247',
 'hsa10249',
 'hsa1025',
 'hsa10269',
 'hsa102724428',
 'hsa102724560',
 'hsa10279',
 'hsa10295',
 'hsa10298',
 'hsa10327',
 'hsa1033',
 'hsa10352',
 'hsa1036',
 'hsa10396',
 'hsa10404',
 'hsa10461',
 'hsa10465',
 'hsa10536',
 'hsa10549',
 'hsa10558',
 'hsa1056',
 'hsa10606',
 'hsa10645',
 'hsa1066',
 'hsa10667',
 'hsa10682',
 'hsa106821730',
 'hsa107',
 'hsa10720',
 'hsa10724',
 'hsa10728',
 'hsa10733',
 'hsa10746',
 'hsa10747',
 'hsa10769',
 'hsa10797',
 'hsa108',
 'hsa1080',
 'hsa10825',
 'hsa10841',
 'hsa10846',
 'hsa10858',
 'hsa10863',
 'hsa10873',
 'hsa10893',
 'hsa109',
 'hsa10901',
 'hsa10924',
 'hsa10935',
 'hsa10941',
 'hsa10988

In [288]:
np.array(e_labels.tolist())

array(['hsa10', 'hsa100', 'hsa10000', 'hsa10013', 'hsa100130827',
       'hsa10014', 'hsa10038', 'hsa10039', 'hsa100526835', 'hsa100533105',
       'hsa10056', 'hsa100861540', 'hsa10105', 'hsa10114', 'hsa10135',
       'hsa10157', 'hsa1017', 'hsa1018', 'hsa10188', 'hsa1019', 'hsa102',
       'hsa1020', 'hsa1021', 'hsa1022', 'hsa1024', 'hsa10243', 'hsa10247',
       'hsa10249', 'hsa1025', 'hsa10269', 'hsa102724428', 'hsa102724560',
       'hsa10279', 'hsa10295', 'hsa10298', 'hsa10327', 'hsa1033',
       'hsa10352', 'hsa1036', 'hsa10396', 'hsa10404', 'hsa10461',
       'hsa10465', 'hsa10536', 'hsa10549', 'hsa10558', 'hsa1056',
       'hsa10606', 'hsa10645', 'hsa1066', 'hsa10667', 'hsa10682',
       'hsa106821730', 'hsa107', 'hsa10720', 'hsa10724', 'hsa10728',
       'hsa10733', 'hsa10746', 'hsa10747', 'hsa10769', 'hsa10797',
       'hsa108', 'hsa1080', 'hsa10825', 'hsa10841', 'hsa10846', 'hsa10858',
       'hsa10863', 'hsa10873', 'hsa10893', 'hsa109', 'hsa10901',
       'hsa10924', 'hsa1

In [298]:
e_drugs, e_labels, e_all_pd, e_x_t_np, e_y_t_np = generate_dti_data_with_drug_feature(e_dti_data, e_fps_with_smi_id)
assert (e_labels.tolist() == e_all_pd.columns[-1411:]).all()

# pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
# np.savez_compressed(pwd + 'E_U_T_xy_np_rdkit_morgan_radius2.npz', x_t_np = e_x_t_np, y_t_np = e_y_t_np)

# np.array(e_labels.tolist())
np.array(e_drugs.tolist())

(1777, 3459)
(1777, 2046) (1777, 1411)
X.shape = (1777, 2046), Y.shape = (1777, 1411)


array(['D11523', 'D07981', 'D10189', 'D01240', 'D10381', 'D00045',
       'D04646', 'D05407', 'D11535', 'D03234', 'D09347', 'D03942',
       'D09950', 'D03643', 'D08484', 'D00292', 'D02349', 'D08838',
       'D04987', 'D10858', 'D05033', 'D00214', 'D05380', 'DB05260',
       'D08024', 'D10926', 'D07136', 'D03883', 'D06167', 'DB01143',
       'D00569', 'D07143', 'D01572', 'D00414', 'D08616', 'D04092',
       'D01630', 'D00917', 'D08061', 'D02896', 'D03765', 'D10730',
       'D00023', 'D01344', 'D11633', 'D05990', 'D02168', 'D08423',
       'D03741', 'D10862', 'D11206', 'D02267', 'D10001', 'D08555',
       'D01118', 'D02341', 'D00369', 'DB13955', 'D05522', 'D08524',
       'D10024', 'D00900', 'D08955', 'D08478', 'D01619', 'D00805',
       'D10317', 'D07193', 'D02368', 'D00362', 'D03252', 'D00881',
       'D11580', 'D02229', 'D08907', 'DB03017', 'D07941', 'D10652',
       'DB04832', 'D01238', 'D00218', 'D06674', 'D10871', 'D00544',
       'D02563', 'D00691', 'D05215', 'D00297', 'D00762', 

In [320]:
# pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
# np.savez_compressed(pwd + 'v2_GPCR_U_T_xy_np_rdkit_morgan_radius2.npz', 
#                     x_t_np = gpcr_x_t_np, 
#                     y_t_np = gpcr_y_t_np,
#                     drug_ids = np.array(drugs),
#                     target_ids = np.array(labels))

ic_drugs, ic_labels, ic_all_pd, ic_x_t_np, ic_y_t_np = generate_dti_data_with_drug_feature(ic_dti_data, ic_fps_with_smi_id)
assert (ic_labels.tolist() == ic_all_pd.columns[-238:]).all()
pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'v2_IC_U_T_xy_np_rdkit_morgan_radius2.npz', 
                    x_t_np = ic_x_t_np, 
                    y_t_np = ic_y_t_np,
                    drug_ids = np.array(ic_drugs),
                    target_ids = np.array(ic_labels))

e_drugs, e_labels, e_all_pd, e_x_t_np, e_y_t_np = generate_dti_data_with_drug_feature(e_dti_data, e_fps_with_smi_id)
assert (e_labels.tolist() == e_all_pd.columns[-1411:]).all()
pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'v2_E_U_T_xy_np_rdkit_morgan_radius2.npz', 
                    x_t_np = e_x_t_np, 
                    y_t_np = e_y_t_np,
                    drug_ids = np.array(e_drugs),
                    target_ids = np.array(e_labels))

(765, 2097)
(765, 1857) (765, 238)
X.shape = (765, 1857), Y.shape = (765, 238)
(1777, 3459)
(1777, 2046) (1777, 1411)
X.shape = (1777, 2046), Y.shape = (1777, 1411)


### MG4 

In [310]:
gpcr_mg4_drugs, gpcr_mg4_labels, gpcr_mg4_all_pd, gpcr_mg4_x_t_np, gpcr_mg4_y_t_np = generate_dti_data_with_drug_feature(gpcr_dti_data, gpcr_mg4_with_smi_id)
assert (gpcr_mg4_labels.tolist() == gpcr_mg4_all_pd.columns[-len(gpcr_mg4_labels):]).all()

pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'GPCR_U_T_xy_np_rdkit_morgan_radius4.npz', 
                    x_t_np = gpcr_x_t_np, 
                    y_t_np = gpcr_y_t_np,
                    drug_ids = np.array(gpcr_mg4_drugs.tolist()),
                    target_ids = np.array(gpcr_mg4_labels.tolist()))

gpcr_mg4_labels.tolist()

(1680, 2206)
(1680, 2048) (1680, 156)
X.shape = (1680, 2048), Y.shape = (1680, 156)


['hsa10161',
 'hsa10203',
 'hsa10266',
 'hsa10267',
 'hsa10268',
 'hsa10800',
 'hsa10803',
 'hsa11251',
 'hsa11255',
 'hsa1128',
 'hsa1129',
 'hsa1131',
 'hsa1132',
 'hsa1133',
 'hsa1234',
 'hsa1241',
 'hsa1268',
 'hsa1269',
 'hsa134',
 'hsa134864',
 'hsa135',
 'hsa136',
 'hsa1394',
 'hsa140',
 'hsa146',
 'hsa147',
 'hsa148',
 'hsa150',
 'hsa151',
 'hsa151306',
 'hsa152',
 'hsa153',
 'hsa154',
 'hsa155',
 'hsa1812',
 'hsa1813',
 'hsa1814',
 'hsa1815',
 'hsa1816',
 'hsa185',
 'hsa186',
 'hsa1901',
 'hsa1903',
 'hsa1909',
 'hsa1910',
 'hsa2149',
 'hsa222545',
 'hsa2357',
 'hsa23620',
 'hsa2550',
 'hsa2692',
 'hsa2693',
 'hsa2740',
 'hsa2798',
 'hsa2835',
 'hsa2841',
 'hsa2846',
 'hsa2847',
 'hsa2852',
 'hsa2859',
 'hsa2862',
 'hsa2864',
 'hsa2911',
 'hsa2912',
 'hsa2913',
 'hsa2914',
 'hsa2915',
 'hsa2916',
 'hsa2917',
 'hsa2918',
 'hsa3061',
 'hsa3062',
 'hsa3269',
 'hsa3274',
 'hsa3350',
 'hsa3351',
 'hsa3352',
 'hsa3354',
 'hsa3355',
 'hsa3356',
 'hsa3357',
 'hsa3358',
 'hsa3360',
 'h

In [311]:
ic_mg4_drugs, ic_mg4_labels, ic_mg4_all_pd, ic_mg4_x_t_np, ic_mg4_y_t_np = generate_dti_data_with_drug_feature(ic_dti_data, ic_mg4_with_smi_id)
assert (ic_mg4_labels.tolist() == ic_mg4_all_pd.columns[-len(ic_mg4_labels):]).all()

pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'IC_U_T_xy_np_rdkit_morgan_radius4.npz', 
                    x_t_np = ic_x_t_np, 
                    y_t_np = ic_y_t_np,
                    drug_ids = np.array(ic_mg4_drugs.tolist()),
                    target_ids = np.array(ic_mg4_labels.tolist()))

ic_mg4_labels.tolist()

(765, 2288)
(765, 2048) (765, 238)
X.shape = (765, 2048), Y.shape = (765, 238)


['hsa10008',
 'hsa10021',
 'hsa10060',
 'hsa10242',
 'hsa10368',
 'hsa10369',
 'hsa1080',
 'hsa11254',
 'hsa11280',
 'hsa1134',
 'hsa1135',
 'hsa1136',
 'hsa1137',
 'hsa1138',
 'hsa1139',
 'hsa1140',
 'hsa1141',
 'hsa1142',
 'hsa1143',
 'hsa1144',
 'hsa1145',
 'hsa1146',
 'hsa116443',
 'hsa116444',
 'hsa1179',
 'hsa1181',
 'hsa1187',
 'hsa1192',
 'hsa1259',
 'hsa162514',
 'hsa169522',
 'hsa170572',
 'hsa200909',
 'hsa200959',
 'hsa22802',
 'hsa22953',
 'hsa23630',
 'hsa23704',
 'hsa2554',
 'hsa2555',
 'hsa2556',
 'hsa2557',
 'hsa2558',
 'hsa2559',
 'hsa2560',
 'hsa2561',
 'hsa2562',
 'hsa2563',
 'hsa2564',
 'hsa2565',
 'hsa2566',
 'hsa2567',
 'hsa2568',
 'hsa2569',
 'hsa2570',
 'hsa26251',
 'hsa27012',
 'hsa27091',
 'hsa27092',
 'hsa27094',
 'hsa27133',
 'hsa27345',
 'hsa2741',
 'hsa2742',
 'hsa2743',
 'hsa285242',
 'hsa2890',
 'hsa2891',
 'hsa2892',
 'hsa2893',
 'hsa2894',
 'hsa2895',
 'hsa2897',
 'hsa2898',
 'hsa2899',
 'hsa2900',
 'hsa2901',
 'hsa2902',
 'hsa2903',
 'hsa2904',
 'hsa

In [312]:
e_mg4_drugs, e_mg4_labels, e_mg4_all_pd, e_mg4_x_t_np, e_mg4_y_t_np = generate_dti_data_with_drug_feature(e_dti_data, e_mg4_with_smi_id)
assert (e_mg4_labels.tolist() == e_mg4_all_pd.columns[-len(e_mg4_labels):]).all()

pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
np.savez_compressed(pwd + 'E_U_T_xy_np_rdkit_morgan_radius4.npz', 
                    x_t_np = e_x_t_np, 
                    y_t_np = e_y_t_np,
                    drug_ids = np.array(e_mg4_drugs.tolist()),
                    target_ids = np.array(e_mg4_labels.tolist()))

e_mg4_labels.tolist()

(1777, 3461)
(1777, 2048) (1777, 1411)
X.shape = (1777, 2048), Y.shape = (1777, 1411)


['hsa10',
 'hsa100',
 'hsa10000',
 'hsa10013',
 'hsa100130827',
 'hsa10014',
 'hsa10038',
 'hsa10039',
 'hsa100526835',
 'hsa100533105',
 'hsa10056',
 'hsa100861540',
 'hsa10105',
 'hsa10114',
 'hsa10135',
 'hsa10157',
 'hsa1017',
 'hsa1018',
 'hsa10188',
 'hsa1019',
 'hsa102',
 'hsa1020',
 'hsa1021',
 'hsa1022',
 'hsa1024',
 'hsa10243',
 'hsa10247',
 'hsa10249',
 'hsa1025',
 'hsa10269',
 'hsa102724428',
 'hsa102724560',
 'hsa10279',
 'hsa10295',
 'hsa10298',
 'hsa10327',
 'hsa1033',
 'hsa10352',
 'hsa1036',
 'hsa10396',
 'hsa10404',
 'hsa10461',
 'hsa10465',
 'hsa10536',
 'hsa10549',
 'hsa10558',
 'hsa1056',
 'hsa10606',
 'hsa10645',
 'hsa1066',
 'hsa10667',
 'hsa10682',
 'hsa106821730',
 'hsa107',
 'hsa10720',
 'hsa10724',
 'hsa10728',
 'hsa10733',
 'hsa10746',
 'hsa10747',
 'hsa10769',
 'hsa10797',
 'hsa108',
 'hsa1080',
 'hsa10825',
 'hsa10841',
 'hsa10846',
 'hsa10858',
 'hsa10863',
 'hsa10873',
 'hsa10893',
 'hsa109',
 'hsa10901',
 'hsa10924',
 'hsa10935',
 'hsa10941',
 'hsa10988